In [1]:
import numpy as np
# from skimage import io
# from matplotlib import pyplot as plt
import tensorflow as tf
import tensorlayer as tl

In [2]:
sess = tf.InteractiveSession()

In [3]:
def rgb2mono(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    # mono = 0.2989 * r + 0.5870 * g + 0.1140 * b
    mono = 0.5 * r + 0.25 * g + 0.25 * b
    return mono

In [4]:
def prepareDataArrays(iterator):
    X = np.zeros(19200)
    y = np.zeros(1)
    recordCounter = 0;

    for string_record in iterator:
        recordCounter += 1

        example = tf.train.Example()
        example.ParseFromString(string_record)
        imageString = (example.features.feature['image'].bytes_list.value[0])
        label = (example.features.feature['label'].int64_list.value[0])

        image = np.fromstring(imageString, dtype=np.uint8)
        image = image.reshape((120, 160, 3))
        image = rgb2mono(image)
        image = image.reshape((19200))

        X = np.vstack((X,image))
        y = np.append(y,label)
        if recordCounter % 100 == 0:
            print(recordCounter,end = ' ')

    y = y.reshape((recordCounter + 1,))
    y = np.round(y / 7) #Downsampling
    y = y + 6
    return X, y

In [5]:
trainIterator = tf.python_io.tf_record_iterator(path="train.tfrecords")
valIterator = tf.python_io.tf_record_iterator(path="val.tfrecords")
testIterator = tf.python_io.tf_record_iterator(path="test.tfrecords")

In [6]:
print("\nTrain...")
X_train, y_train = prepareDataArrays(trainIterator)
print("\nVal...")
X_val, y_val = prepareDataArrays(valIterator)
print("\nTest...")
X_test, y_test = prepareDataArrays(testIterator)


Train...


D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 
Val...
100 200 300 400 500 600 700 800 900 1000 
Test...
100 200 300 400 500 600 700 800 900 1000 

In [7]:
x = tf.placeholder(tf.float32, shape=[None, 19200], name='x')
y_ = tf.placeholder(tf.int64, shape=[None], name='y_')

In [8]:
network = tl.layers.InputLayer(x, name='input')
network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
network = tl.layers.DenseLayer(network, 4096, tf.nn.relu, name='relu1')
# network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
network = tl.layers.DenseLayer(network, 1024, tf.nn.relu, name='relu2')
# network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
network = tl.layers.DenseLayer(network, n_units=13, act=tf.identity, name='output')

[TL] InputLayer  input: (?, 19200)
[TL] DropoutLayer drop1: keep:0.800000 is_fix:False
[TL] DenseLayer  relu1: 4096 relu
[TL] DenseLayer  relu2: 1024 relu
[TL] DenseLayer  output: 13 identity


In [9]:
y = network.outputs
cost = tl.cost.cross_entropy(y, y_, name='cost')
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
y_op = tf.argmax(tf.nn.softmax(y), 1)

In [10]:
train_params = network.all_params
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost, var_list=train_params)

In [13]:
tl.layers.initialize_global_variables(sess)

ResourceExhaustedError: OOM when allocating tensor with shape[19200,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: relu1/W/Adam/Initializer/zeros = Fill[T=DT_FLOAT, _class=["loc:@relu1/W"], index_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](relu1/W/Adam/Initializer/zeros/shape_as_tensor, relu1/W/Adam/Initializer/zeros/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'relu1/W/Adam/Initializer/zeros', defined at:
  File "D:\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Anaconda3\envs\tensorf

In [ ]:
network.print_params()

In [ ]:
network.print_layers()

In [18]:
tl.utils.fit(
    sess, network, train_op, cost, X_train, y_train, x, y_, acc=acc, batch_size=50, n_epoch=500, print_freq=5, X_val=X_val, y_val=y_val, eval_train=False)

[TL] Start training the network ...
[TL] Epoch 1 of 500 took 5.168405s
[TL]    val loss: 144.880801
[TL]    val acc: 0.391000
[TL] Epoch 5 of 500 took 2.822076s
[TL]    val loss: 2.198568
[TL]    val acc: 0.197000
[TL] Epoch 10 of 500 took 2.858738s
[TL]    val loss: 1.977089
[TL]    val acc: 0.197000
[TL] Epoch 15 of 500 took 2.844629s
[TL]    val loss: 1.887335
[TL]    val acc: 0.197000
[TL] Epoch 20 of 500 took 2.859026s
[TL]    val loss: 1.843414
[TL]    val acc: 0.364000
[TL] Epoch 25 of 500 took 2.730402s
[TL]    val loss: 1.817066
[TL]    val acc: 0.365000
[TL] Epoch 30 of 500 took 2.878434s
[TL]    val loss: 1.804229
[TL]    val acc: 0.365000
[TL] Epoch 35 of 500 took 2.842181s
[TL]    val loss: 1.795692
[TL]    val acc: 0.364000
[TL] Epoch 40 of 500 took 2.817311s
[TL]    val loss: 1.788771
[TL]    val acc: 0.365000
[TL] Epoch 45 of 500 took 2.836905s
[TL]    val loss: 1.785212
[TL]    val acc: 0.364000
[TL] Epoch 50 of 500 took 2.713323s
[TL]    val loss: 1.780157
[TL]    val

KeyboardInterrupt: 

In [15]:
tl.utils.test(sess, network, acc, X_test, y_test, x, y_, batch_size=None, cost=cost)

[TL] Start testing the network ...


ValueError: Cannot feed value of shape (19200, 1) for Tensor 'x:0', which has shape '(?, 19200)'

In [16]:
tl.files.save_npz(network.all_params, name='model.npz')
sess.close()

[TL] [*] model.npz saved
